# LAV-ESG fileserver access: getting started

This is a jupyter notebook intended for all students working with the fileserver data of the Energy Systems Group of LAV. In this short document you should get a first impression of what a notebook file is, how it looks like and you should learn how to connect to the fileserver to access data.

## About jupyter notebook

It is an environment consisting of a **cell based** structure allowing to combine documentation (Markdown) and coding. Each cell can be accessed by *double click* and executed by pressing *Shift + Enter* or *Ctrl + Enter*.

There are many cool things possible, like:
- bulleting
- $latex$
- and its math tool: $ e^{\pi \cdot i} + 1 = 0$
- even centered $$ e^{\pi \cdot i} + 1 = 0$$
- references and links to files or webpages. You find the official [description here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/index.html). Just check it out.

There are many [tutorials](https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Index.ipynb) online - either directly from *Jupyter*, provided by github or tutorial videos. Just google for them. After you went through this notebook it is recommended to start with a tutorial. 

Of course you can execute some python code directly in the notebook (or load it from an external source). Make yourself familiar with shortcuts to quickly switch between *Code* and *Markdown* cells. You can see the list of shortcuts when you press *h*. Knowing this, we finally want to come up with some coding! First, let us import a package and rename it (you need to execute the cells below, having *IN [ ]* in front of them):

In [ ]:
import numpy as np

This package allows us for instance to perform array multiplication (element-wise):

In [ ]:
# define the arrays
a = np.array([2, 4, 6])
b = np.array([1, 2, 1])

In [ ]:
c = a*b

We can see the result by printing it (or giving it directly)

In [ ]:
c

In [ ]:
print(c)

And it acutally works. This should just be a trivial example to let you see how those cells work AND that packages can be imported. Such a package is the one you just installed in STEP 3 (point 2) of the *LAV-ESG_gettingStarted* document. This package provides useful commands to access the fileserver and its data. Let's see how it works...

## Access the LAV-fileserver

First (again), import the needed package (LAV extension) by executing the cell below:

In [ ]:
%load_ext ipython_pg.ipython_extension_LAV

and connect to the server, namely the *mobility* database (**HERE** you are asked to enter your password. You need your separate password from your supervisor. It is important not to have access passwords stored anywhere in a code file or notebook, but use this line magic to connect to the database):

Enter your username below (replace 'xxxxxxxxx') before executing the cell

In [ ]:
%pg_connect host='lav-fileserver' user='xxxxxxxxx' port='5433' dbname='mobility' sslcert=sslcert

If you installed everything correctly, there should be the line:

    SUCCESS: connected to lav-fileserver

Now we can access the data with *queries*, which return *cursors* pointing to the data. Below you will find some examples how it can be done. Run them and try to understand how they work. If you need help with the *SQL* language, search for tutorials or documentation online.

For help regarding the LAV extension, you can see magic documentation if you add a *?* after the command. To actually found out what different magics are available, click in the cell below (cursor should be at the end) and press *TAB*. There should appear a drop down menu with all the magics available. Select one (e.g. *%pg_cursor*) by using the 'arrow down' of your keyboard and press enter. Now add a *?* and execute the cell. The description of the magic should appear at the bottom.

In [ ]:
%pg_

An alternative to the *pgAdmin* software is the *%pg_info* magic. It provides infromation regarding the dataset structure including their description. You can check it out by executing the cell below.

In [ ]:
%pg_info

**NOTE**: to execute the sql queries below you require the correct *access roles* for your login. You won't be able to get data if you can't access the tables. You signed a NDA which grants you access to specific schemas. To proceed with this notebook you need permissions for the **freight schemas**, namely **GTE, LWE, GQGV and LSVA**. To see what those schema contain, we can again run the *%pg_info* magic and add the name of the schema at the end.

for GTE:

In [ ]:
%pg_info GTE

for LWE:

In [ ]:
%pg_info LWE

for GQGV:

In [ ]:
%pg_info GQGV

and for LSVA:

In [ ]:
%pg_info LSVA

### cell magic with cursor

The cell magic (*%%pg_sql*) allows you to write your query directly in one cell and execute it. In the example below, we are accessing the schema *GTE* (Gütertransporterhebung) and within it the table *week2013*. The data we are *selecting* is the *cantonOfOrigin* (Origin of a vehicle) and *count* how many vehicles are contained in the survey of each canton. To do that, we need to *group* the data for each *cantonOfOrigin*. The word after the cell magic is the name of the cursor which will be returned, here: *cur*

In [ ]:
%%pg_sql cur
SELECT w."cantonOfOrigin", count(*) as amount FROM "GTE".week2013 w GROUP BY w."cantonOfOrigin" ORDER BY amount desc

If everything works properly you should see:

    SUCESS: matched 26 rows
     cursor object as 'cur'
   
This means we got a cursor returned consisting of 26 rows. We can now extract the data by iterating the cursor, and e.g. store it as tuples in a list:

In [ ]:
cantonCount = []
for row in cur:
    cantonCount.append((row[0], row[1]))

If you want to see how the *cantonCount* list looks like, we can print its content

In [ ]:
for item in cantonCount:
    print("In canton {:2s}, {: 4d} vehicles took part in the survey".format(item[0], item[1]))

This should just be an example how you access the data and then further process it within your python code. Your query will be most likely more complexe.

### cell magic without cursor

In the example above, we basically just wanted to see the result of the query and are not using the data any further. With the LAV extension this can be done directly, if you don't return a cursor. Thus leave away the word after the cell magic and the results of the query will get illustrated

In [ ]:
%%pg_sql
SELECT w."cantonOfOrigin", count(*) as amount FROM "GTE".week2013 w GROUP BY w."cantonOfOrigin" ORDER BY amount desc

See, we got the exact same outcome directly. This is useful when you're just interessted in investigating the data. You can do that with any kind of query. E.g. below we spice it up a bit, where we look into the performance of the vehicles of each canton, i.e. how many goods (in ton) they transported over distance(tkms). For this we need to know what good was transported with which vehicle. This information is contained in a separate table which we need to access. We start with the original querry above (outter query) and join the second data tables on the identical variable, here the OID of the vehicle (*"ernr"*). Because multiple goods can be transported with just one vehicle, we need to compute the performance (tkm) of a single vehicle by grouping over all good entries of that vehicle (inner query). The *LEFT JOIN* command allows the outter query to access this performance value for each vehicle. This value is multiplied with the weighting factor of the *week2013* table and summed up for all vehicles within one canton. The fourth colum is just the vehicle average. Got it? Just run the cell and see what happens

In [ ]:
%%pg_sql
SELECT w."cantonOfOrigin", count(*) as amount, 
       sum(w."whTotCal"*m."tkm")*1e-6 as "mio. tkm", 
       sum(w."whTotCal"*m."tkm")*1e-6/count(*) as "mio. tkm / veh"
FROM "GTE".week2013 w 
LEFT JOIN ( SELECT
          t."ernr",
          sum(t."tonCH"*1e-3*t."transportKmCH") as tkm
          FROM "GTE".transport2013 as t
          GROUP by t."ernr") m ON w."ernr"=m."ernr"
GROUP BY w."cantonOfOrigin" 
ORDER BY amount desc

Again, it's just an example. Nevertheless, it should help you to get the picture how you can use this extension. Of course you don't have to use the cell magic, but can execute queries based on strings. An example of this method is shown below.

### sql string

We can run the same query from above, listing the query as a sequence of strings:

In [ ]:
sql = ('SELECT w."cantonOfOrigin", count(*) as amount',
       'FROM "GTE".week2013 w',
       'GROUP BY w."cantonOfOrigin"',
       'ORDER BY amount desc'
      )

try:
    cur = %pg_cursor
    cur.execute(" ".join(sql))
except Exception as e:
    print(str(e))
    %pg_rollback()

And print results just as above:

In [ ]:
for item in cur:
    print("In canton {:2s}, {: 4d} vehicles took part in the survey".format(item[0], item[1]))

how you segment (or if) is up to you. It can bring some advantages with it though. The advantage of using the string form to run queries is the easy modification of the sql to run similar queries in one cell, like the one below. Here we simply start from the more complex performance query, but we only want to execute it for several cantons.

In [ ]:
sql = ('SELECT w."cantonOfOrigin", count(*) as amount,',
       'sum(w."whTotCal"*m."tkm")*1e-6 as "mio. tkm",',               # tkm of canton
       'sum(w."whTotCal"*m."tkm")*1e-6/count(*) as "mio. tkm / veh"'  # average vehicle tkm of canton
       'FROM "GTE".week2013 w ',
       'LEFT JOIN ( SELECT t."ernr", sum(t."tonCH"*1e-3*t."transportKmCH") as tkm', # vehicle tkm of transport table
       'FROM "GTE".transport2013 as t GROUP by t."ernr") m ON w."ernr"=m."ernr"'    # left join
       'WHERE w."cantonOfOrigin"',  #  offene Bedingung (= *canton* fehlt)
      )
sql = " ".join(sql)

for text, cond in (("Zurich","='ZH'"), ("Lucerne","='LU'"), ("Schwyz","='SZ'")):
        
    try:
        cur = %pg_cursor
        cur.execute(" ".join((sql, cond, 'GROUP BY w."cantonOfOrigin"')))
        print("canton of {:8s} code {:2s}, {:3d} veh, {:>7.2f} mio. tkm, {:1.2f} mio. tkm / vkm".format(
                "{}:".format(text), *cur.fetchone()))
    except Exception as e:
        print(str(e))
        %pg_rollback()

This is the end of the gettingStarted notebook. We hope we were able to show you the functioning principle and usability of the jupyter notebook in combination with the LAV extension to access the LAV fileserver. It is up to you what environment you are using, as long as you are able to run sql queries. Good luck and have fun with your project! Even though you might not have access to the other schemas, you are welcome to take a look at the other notebooks to see different *sql querries*.